# Integração entre as Bases RD Station e Exact Sales

> Aqui vamos apontar e resolver os problemas de integração entre as bases [RD Station](https://plugcrm.net/) e [Exact Sales](https://app.exactspotter.com/Account/NormalLogin?email=fernando%40absolutasaude.com.br), duas bases de CRM utilizadas pela empresa [Absoluta Saúde](https://absolutasaude.com.br/)

## O Problema

O problema a ser resolvido está relacionado à importação da base de _leads_ da base RD para a base Exact, e são eles:

- **Importação incompleta do número de _leads_**: não foram importados todos os leads para a base Exact, sendo que temos 18197 _leads_ na base Exact e 32346 _leads_ na base RD. Assim, _temos 14149 leads que não foram importados para a base Exact_;
- **Importação incompleta das informações dos _leads_**: alguns _leads_ não foram importados com todas as informações para a base Exact, mesmo tendo essas informações na base RD. Alguns casos como: ZANON TECNOLOGIA LTDA ME, VIDAM BRASIL AG DE TURISMO E VIAGENS LTDA entre outras.
    - Uma possível causa desse problema é que pelo formato de saída da exportação do RD do arquivo `.json` ter o _array_ de objetos `custom_fields`, o Exact pode não reconhecer isso da melhor forma na hora da importação

## [Exact Sales](https://app.exactspotter.com/Account/NormalLogin?email=fernando%40absolutasaude.com.br)

> Temos **18197 leads** nessa base

> - Login: fernando@absolutasaude.com.br
> - Senha: 0i9gky32

### API

- Manual: [Exact Spotter V2 - API v3](https://exactsal.es/apiv3-spotterv2)
- Chave: https://api.exactspotter.com/v3/Leads
- Token: 35fc1bd4-7d6e-4f40-bffd-123d2f46054a

#### Limitações
- Limite de 500 leads por requisição
- Limite de 30 requisições a cada 20 segundos

In [ ]:
# GET Request para leads da Exact

import requests

headers = {
    'Content-Type': 'application/json',
    'token_exact': '35fc1bd4-7d6e-4f40-bffd-123d2f46054a'
}

response = requests.get(
    'https://api.exactspotter.com/v3/Leads', headers=headers)

print(response.status_code)
# print(response.headers)
print(response.json())


In [ ]:
# Criando base de dados para o Exact

import requests
import json

pagesErrorExct = []
totalLeadsExct = 0

# Funções
# Função para a requisição da API


def request_API(skipNum):
    url = 'https://api.exactspotter.com/v3/Leads'
    headers = {
        'Content-Type': 'application/json',
        'token_exact': '35fc1bd4-7d6e-4f40-bffd-123d2f46054a'
    }
    params = {
        '$skip': skipNum,
        '$orderby': 'lead'
    }

    result = requests.get(url, headers=headers, params=params)

    # Se a requisição for um sucesso, retorna o resultado
    if (result.status_code == 200):
        print('Requisição: Sucesso')
        return result

    # Se a requisição falhar, adiciona o número da página à lista de erro
    elif (result.status_code == 400):
        print('Requisição: Falhou')
        # pagesErrorExct.append(page)

# Função para remover itens da lista


def remove_items(list):
    for i in range(0, len(list)):
        # Contando quantos leads tem nessa lista
        global totalLeadsExct
        totalLeadsExct = totalLeadsExct + 1

        # Removendo campos desnecessários de cada lead
        # list[i].pop('registerDate')
        list[i].pop('updateDate')
        list[i].pop('mktLink')
        list[i].pop('leadProduct')
        list[i].pop('stage')
        list[i].pop('integrationId')

        # Corrigindo o número de dígitos do CNPJ
        lenCNPJ = len(list[i]["cnpj"])
        if (lenCNPJ < 14):
            if (lenCNPJ == 13):
                list[i]["cnpj"] = "0" + list[i]["cnpj"]
            elif (lenCNPJ == 12):
                list[i]["cnpj"] = "00" + list[i]["cnpj"]
            elif (lenCNPJ == 11):
                list[i]["cnpj"] = "000" + list[i]["cnpj"]
            elif (lenCNPJ == 10):
                list[i]["cnpj"] = "0000" + list[i]["cnpj"]
            elif (lenCNPJ == 9):
                list[i]["cnpj"] = "00000" + list[i]["cnpj"]
            elif (lenCNPJ == 8):
                list[i]["cnpj"] = "000000" + list[i]["cnpj"]

            # Leads sem CNPJ
            # elif (lenCNPJ == 0):
                # print(list[i]['lead'])

            # else:
                # if (lenCNPJ != 0):
                # print(list[i]["cnpj"])
                # print(lenCNPJ)


# Criando a database
listCompleteExct = []

# Usando um número bem alto como 50000 para não correr o risco de não trazer todos os leads
for i in range(0, 30000, 500):
    print('Skip atual:', i)
    # Atualizando a requisição
    responseExct = request_API(i)

    if (responseExct):
        # Transformando em lista (array)
        responseStrExct = json.dumps(responseExct.json()["value"])
        listExct = json.loads(responseStrExct)

        if (listExct):
            print('Skip tem leads')
            # Removendo alguns itens da lista
            remove_items(listExct)

            # Juntando as listas
            listCompleteExct.append(listExct)
            jsonDatabaseExct = json.dumps(listCompleteExct)

        else:
            print('Skip não tem leads')

# Escrevendo no arquivo json
jsonFileExct = open("completeDatabase_Exact2.json", "w")
jsonFileExct.write(jsonDatabaseExct)
jsonFileExct.close()

print('Páginas com erro:', pagesErrorExct)
print('Total de leads:', totalLeadsExct)
print("Finalizado")


## Excluindo Leads Duplicados Exact

> Aqui vamos fazer excluir os leads com o mesmo CNPJ da base do Exact Sales

### Procedimento

- Buscamos os leads com o CNPJ duplicado na base exact e excluímos os com menos informação, deixando somente um


In [ ]:
# Excluindo os leads duplicados da base Exact

# Lendo a base Exact
listExctOpen = open("completeDatabase_Exact2.json")
listExct = json.load(listExctOpen)

totalLeadsExctDupl = 0
resultExctDupl = []

# Loop correndo para cada CNPJ da base exact, se existe duplicado cria um objeto e insere numa lista
for s in range(0, len(listExct) - 1):
    # for s in range(0, 1):
    print(s)  # 53 no total
    for t in range(0, len(listExct[s])):
        CNPJExct = listExct[s][t]["cnpj"]
        for i in range(0, len(listExct) - 1):
            for j in range(0, len(listExct[i])):
                if ((s, t) != (i, j)):
                    if (listExct[s][t]["cnpj"] and listExct[i][j]["cnpj"]):
                        if (listExct[s][t]["cnpj"] == listExct[i][j]["cnpj"]):
                            resultObj = {
                                'CNPJ': CNPJExct,
                                'Nome Exact': listExct[i][j]["lead"],
                                'Repeticoes': 2
                            }
                            resultExctDupl.append(resultObj)

# Loop para limpar os duplicados mais de uma vez
for i in range(0, len(resultExctDupl) - 1):
    try:
        for j in range(0, len(resultExctDupl)):
            if (i != j):
                if (resultExctDupl[j]["CNPJ"] == resultExctDupl[i]["CNPJ"]):
                    resultExctDupl[j]["Repeticoes"] = resultExctDupl[j]["Repeticoes"] + 1
                    del resultExctDupl[i]
                    break
    except:
        break
for i in range(0, len(resultExctDupl) - 1):
    try:
        for j in range(0, len(resultExctDupl)):
            if (i != j):
                if (resultExctDupl[j]["CNPJ"] == resultExctDupl[i]["CNPJ"]):
                    resultExctDupl[j]["Repeticoes"] = resultExctDupl[j]["Repeticoes"] + 1
                    del resultExctDupl[i]
                    break
    except:
        break

# Loop para contar quantos duplicados exatos existem na base
for i in range(0, len(resultExctDupl)):
    totalLeadsExctDupl = totalLeadsExctDupl + 1

# Loop para excluir os duplicados da base Exact
deletedLeads = []
totalDeletedLeads = 0

for i in range(0, len(resultExctDupl) - 1):
    control = False
    try:
        for j in range(0, len(listExct) - 1):
            for k in range(0, len(listExct[j])):
                if (resultExctDupl[i]["CNPJ"] and listExct[j][k]["cnpj"]):
                    if (resultExctDupl[i]["CNPJ"] == listExct[j][k]["cnpj"]):
                        deletedLeads.append(resultExctDupl[i]["CNPJ"])
                        del listExct[j][k]
                        control = True
                        break
                if (control == True):
                    break
            if (control == True):
                break
    except:
        break

for i in range(0, len(resultExctDupl) - 1):
    control = False
    try:
        for j in range(0, len(listExct) - 1):
            for k in range(0, len(listExct[j])):
                if (resultExctDupl[i]["CNPJ"] and listExct[j][k]["cnpj"]):
                    if (resultExctDupl[i]["CNPJ"] == listExct[j][k]["cnpj"]):
                        deletedLeads.append(resultExctDupl[i]["CNPJ"])
                        del listExct[j][k]
                        control = True
                        break
                if (control == True):
                    break
            if (control == True):
                break
    except:
        break

# Loop para contar quantos duplicados exatos foram excluídos da base
for i in range(0, len(listExct)):
    totalDeletedLeads = totalDeletedLeads + 1

# Escrevendo para so arquivos json
resultExctDuplStr = json.dumps(resultExctDupl)
jsonFileExctFinal = open("ExactDuplicateExact.json", "w")
jsonFileExctFinal.write(resultExctDuplStr)
jsonFileExctFinal.close()

deletedLeadsStr = json.dumps(deletedLeads)
jsonFileDeletedLeads = open("exactDeletedLeads.json", "w")
jsonFileDeletedLeads.write(deletedLeadsStr)
jsonFileDeletedLeads.close()

listExctFinal = json.dumps(listExct)
jsonFileListExctFinal = open("exactListFinal.json", "w")
jsonFileListExctFinal.write(listExctFinal)
jsonFileListExctFinal.close()

print("Total de leads duplicados Exact: ", totalLeadsExctDupl)
print("Total de leads excluídos Exact: ", totalDeletedLeads)
print("Finalizado")


## [RD Station](https://plugcrm.net/)

> Temos **32346 leads** nessa base

> - Login: luciane@absolutasaude.com.br
> - Senha: SgAbs76@380

> - Login: f.aspires@hotmail.com
> - Senha: abs020172

### API

- Manual: [Developers RD Station](https://developers.rdstation.com/reference/instruções-e-requisitos)
- Chave: https://crm.rdstation.com/api/v1/organizations?token=5b67dabde7b4bd01146f3d53
- Token: 5b67dabde7b4bd01146f3d53, Conta f.aspires: 5b67dabde7b4bd01146f3d52

#### Limitações

- Limite de 200 leads por requisição
- Limite de 120 requisições por minuto


In [ ]:
# GET Request para leads da RD

import requests

params = {
    'token': '5b67dabde7b4bd01146f3d53'
}

response = requests.get(
    'https://crm.rdstation.com/api/v1/organizations', params=params)

print(response.status_code)
# print(response.headers)
print(response.json())


In [ ]:
# Criando base de dados para o RD

import requests
import json

pagesErrorRd = []
totalLeadsRd = 0

# Funções
# Função para a requisição da API


def request_API(id, page):
    url = 'https://crm.rdstation.com/api/v1/organizations'
    params = {
        'token': '5b67dabde7b4bd01146f3d53',
        'page': page,
        'limit': 500,
        'order': 'name',
        'direction': 'asc',
        'user_id': id
    }

    result = requests.get(url, params=params)

    # Se a requisição for um sucesso, retorna o resultado
    if (result.status_code == 200):
        print('Requisição: Sucesso')
        return result

    # Se a requisição falhar, adiciona o número da página à lista de erro
    elif (result.status_code == 400):
        print('Requisição: Falhou')
        pagesErrorRd.append(page)

# Função para remover itens da lista


def remove_items(list):
    for i in range(0, len(list)):
        # Contando quantos leads tem nessa lista
        global totalLeadsRd
        totalLeadsRd = totalLeadsRd + 1

        # Removendo campos desnecessários de cada lead
        list[i].pop('resume')
        list[i].pop('won_count')
        list[i].pop('lost_count')
        list[i].pop('opened_count')
        list[i].pop('paused_count')
        list[i].pop('updated_at')
        list[i].pop('organization_segments')

        if len(list[i]["contacts"]):
            list[i]["contacts"][0].pop('notes')
            list[i]["contacts"][0].pop('facebook')
            list[i]["contacts"][0].pop('linkedin')
            list[i]["contacts"][0].pop('skype')
            list[i]["contacts"][0].pop('birthday')

        if len(list[i]["custom_fields"]):
            # Limpando o campo CNPJ, deixando somente os números
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CNPJ"):
                    disallowed_characters = "./-"
                    for character in disallowed_characters:
                        if (list[i]["custom_fields"][j]["value"]):
                            list[i]["custom_fields"][j]["value"] = list[i]["custom_fields"][j]["value"].replace(
                                character, "")

                    # Corrigindo o número de dígitos do CNPJ
                    if (list[i]["custom_fields"][j]["value"]):
                        lenCNPJ = len(list[i]["custom_fields"][j]["value"])
                        if (lenCNPJ < 14 and list[i]["custom_fields"][j]["value"]):
                            if (lenCNPJ == 13):
                                list[i]["custom_fields"][j]["value"] = "0" + \
                                    list[i]["custom_fields"][j]["value"]
                            elif (lenCNPJ == 12):
                                list[i]["custom_fields"][j]["value"] = "00" + \
                                    list[i]["custom_fields"][j]["value"]
                            elif (lenCNPJ == 11):
                                list[i]["custom_fields"][j]["value"] = "000" + \
                                    list[i]["custom_fields"][j]["value"]
                            elif (lenCNPJ == 10):
                                list[i]["custom_fields"][j]["value"] = "0000" + \
                                    list[i]["custom_fields"][j]["value"]
                            elif (lenCNPJ == 9):
                                list[i]["custom_fields"][j]["value"] = "00000" + \
                                    list[i]["custom_fields"][j]["value"]
                            elif (lenCNPJ == 8):
                                list[i]["custom_fields"][j]["value"] = "000000" + \
                                    list[i]["custom_fields"][j]["value"]

            # Excluindo campos desnecessários dos custom fields
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "FIXO 2"):
                    del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "FIXO 3"):
                    del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CELULAR 2"):
                    del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CELULAR 3"):
                    del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "EMAIL 2"):
                    del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "EMAIL 3"):
                    del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "COMERCIAL 1"):
                    del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "COMERCIAL 2"):
                    del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "COMERCIAL 3"):
                    del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CEL 2  SOCIO 1"):
                    del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CEL  3  SOCIO 1"):
                    del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "NOME SOCIO 2"):
                    del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CATEGORIA SOCIO 2"):
                    del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CEL 1  SOCIO 2"):
                    del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CEL 2  SOCIO 2"):
                    del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CEL 3  SOCIO 2"):
                    del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "NOME SOCIO 3"):
                    del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CATEGORIA SOCIO 3"):
                    del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CEL 1  SOCIO 3"):
                    del [list[i]["custom_fields"][j]]
            for j in range(0, len(list[i]["custom_fields"]) - 1):
                if (list[i]["custom_fields"][j]["custom_field"]["label"] == "CEL 2  SOCIO 3"):
                    del [list[i]["custom_fields"][j]]


# Usuários
# Tem a requisição para isso, mas não traz todos os usuários
# lenUsersId = 39
usersId = [
    '5aabc9e0276cfa000172b5e5',
    '5ab552396d3909000c2fef1f',
    '5abccec74b081b000e0cb4fd',
    '5ac3970c5a866b000d0ea164',
    '5ac397a23e9bd3000ccd6aef',
    '5ac3980f3e9bd3000ccd6b78',
    '5ac4ceb9975369000ba86002',
    '5ac512b7ff595e001899fbaf',
    '5acced2c3eb788000b6de43c',
    '5ae210d9b96ff8000bbd3a32',
    '5b0db0582ee7c5000b820966',
    '5b23cd1217762e000b8f1ae8',
    '5b23cdb817762e00148f15ba',
    '5b2b9fbeb73caa0018c70f9a',
    '5b3f6f62fdc6ad000b38336d',
    '5c06879c17a5240010a6a562',
    '5c35f0adabcdfc0021b21ff5',
    '5c38757b7fbef70010b93bd4',
    '5c49a36c2f105c002c560354',
    '5c798c6662345d00105988c3',
    '5e15fed90b64410025ae2327',
    '5e31f0a16094d9001071951c',
    '5ecd7e38692a01002956739f',
    '5ecfc1279580a10024a1816f',
    '5ede4820bec78d0024d99daa',
    '5f0cbf70649f1500296d29b4',
    '5f0f0623356df5002b0cfaaa',
    '5f13056866f067000ebdf6da',
    '5f25f890a5281e0019080727',
    '5f6497bfbb41e200207a9011',
    '5fae972f03d7d80023cf9fa4',
    '5fc628bcca2160001e2527fb',
    '5fc637ad204518000afdeafd',
    '5fc6396aec62040013276d5a',
    '5fc63a9f786e270012c78725',
    '5fc658ebffb234000959302c',
    '5fc65dc4eee8cf0015b2dd89',
    '60a2f17e4899a100161e72a9',
    '6123c1a49f809a000b8d4591',
    '6171921d9edde5001fd0bb6f',
]

# Criando a database
listCompleteRd = []

for i in range(0, len(usersId)):
    print('ID:', i + 1)
    # AQUI TEMOS O ERRO DE LIMITAÇÃO DA API, ACIMA DE 50 PÁGINAS COM 500 POR REQUISIÇÃO RECEBEMOS ERRO
    for j in range(1, 50):  # Usando um número bem alto como 50 para não correr o risco de não trazer todos os leads
        print('Página:', j)
        # Atualizando a requisição
        responseRd = request_API(usersId[i], j)

        if (responseRd):
            # Transformando em lista (array)
            responseStrRd = json.dumps(responseRd.json()["organizations"])
            listRd = json.loads(responseStrRd)

            if (listRd):
                print('Página tem leads')

                # Removendo alguns itens da lista
                remove_items(listRd)

                # Juntando as listas
                listCompleteRd.append(listRd)
                jsonDatabaseRd = json.dumps(listCompleteRd)

            else:
                print('Página não tem leads')
                break

# Escrevendo no arquivo json
jsonFileRd = open("completeDatabase_RD2.json", "w")
jsonFileRd.write(jsonDatabaseRd)
jsonFileRd.close()

print('Páginas com erro:', pagesErrorRd)
print('Total de leads:', totalLeadsRd)
print("Finalizado")


## Excluindo Leads Duplicados RD

> Aqui vamos fazer excluir os leads com o mesmo CNPJ da base do RD Station

### Procedimento

- Buscamos os leads com o CNPJ duplicado na base RD e excluímos os com menos informação, deixando somente um


In [ ]:
# Excluindo os leads duplicados da base RD

# Lendo a base RD
listRdOpen = open("completeDatabase_RD2.json")
listRd = json.load(listRdOpen)

totalLeadsRdDupl = 0
resultRdDupl = []

# Loop correndo para cada CNPJ da base RD, se existe duplicado cria um objeto e insere numa lista

# for w in range(0, 1):
for w in range(0, len(listRd)):
    print(w)  # 139
    for s in range(0, len(listRd[w])):
        for t in range(0, len(listRd[w][s]["custom_fields"]) - 1):
            if (listRd[w][s]["custom_fields"][t]["custom_field"]["label"] == "CNPJ"):
                CNPJRd = listRd[w][s]["custom_fields"][t]["value"]
                for z in range(0, len(listRd)):
                    for i in range(0, len(listRd[z])):
                        for j in range(0, len(listRd[z][i]["custom_fields"]) - 1):
                            if ((s, t) != (i, j)):
                                if (listRd[z][i]["custom_fields"][j]["value"] and listRd[w][s]["custom_fields"][t]["value"]):
                                    if (listRd[z][i]["custom_fields"][j]["value"] == listRd[w][s]["custom_fields"][t]["value"]):
                                        resultObj = {
                                            'CNPJ': CNPJRd,
                                            'Nome RD': listRd[z][i]["name"],
                                            'Repeticoes': 2
                                        }
                                        resultRdDupl.append(resultObj)

# Loop para limpar os duplicados mais de uma vez
for i in range(0, len(resultRdDupl) - 1):
    try:
        for j in range(0, len(resultRdDupl)):
            if (i != j):
                if (resultRdDupl[j]["CNPJ"] == resultRdDupl[i]["CNPJ"]):
                    resultRdDupl[j]["Repeticoes"] = resultRdDupl[j]["Repeticoes"] + 1
                    del resultRdDupl[i]
                    break
    except:
        break
for i in range(0, len(resultRdDupl) - 1):
    try:
        for j in range(0, len(resultRdDupl)):
            if (i != j):
                if (resultRdDupl[j]["CNPJ"] == resultRdDupl[i]["CNPJ"]):
                    resultRdDupl[j]["Repeticoes"] = resultRdDupl[j]["Repeticoes"] + 1
                    del resultRdDupl[i]
                    break
    except:
        break

# Loop para contar quantos duplicados exatos existem na base
for i in range(0, len(resultRdDupl)):
    totalLeadsExctDupl = totalLeadsExctDupl + 1

# Loop para excluir os duplicados da base RD
deletedLeads = []
totalDeletedLeads = 0

for i in range(0, len(resultRdDupl) - 1):
    control = False
    try:
        for w in range(0, len(listRd)):
            for s in range(0, len(listRd[w])):
                for t in range(0, len(listRd[w][s]["custom_fields"]) - 1):
                    if (listRd[w][s]["custom_fields"][t]["custom_field"]["label"] == "CNPJ"):
                        if (listRd[w][s]["custom_fields"][t]["value"] == resultRdDupl[i]["CNPJ"]):
                            deletedLeads.append(resultRdDupl[i]["CNPJ"])
                            del listRd[w][s]
                            control = True
                            break
                if (control == True):
                    break
            if (control == True):
                break
    except:
        break

for i in range(0, len(resultRdDupl) - 1):
    control = False
    try:
        for w in range(0, len(listRd)):
            for s in range(0, len(listRd[w])):
                for t in range(0, len(listRd[w][s]["custom_fields"]) - 1):
                    if (listRd[w][s]["custom_fields"][t]["custom_field"]["label"] == "CNPJ"):
                        if (listRd[w][s]["custom_fields"][t]["value"] == resultRdDupl[i]["CNPJ"]):
                            deletedLeads.append(resultRdDupl[i]["CNPJ"])
                            del listRd[w][s]
                            control = True
                            break
                if (control == True):
                    break
            if (control == True):
                break
    except:
        break

# Loop para contar quantos duplicados exatos foram excluídos da base
for i in range(0, len(listRd)):
    totalDeletedLeads = totalDeletedLeads + 1

# Escrevendo para so arquivos json
resultRdDuplStr = json.dumps(resultRdDupl)
jsonFileRdFinal = open("RdDuplicateLeads.json", "w")
jsonFileRdFinal.write(resultRdDuplStr)
jsonFileRdFinal.close()

deletedLeadsStr = json.dumps(deletedLeads)
jsonFileDeletedLeads = open("RdDeletedLeads.json", "w")
jsonFileDeletedLeads.write(deletedLeadsStr)
jsonFileDeletedLeads.close()

listRdFinal = json.dumps(listRd)
jsonFileListRdFinal = open("RdListFinal.json", "w")
jsonFileListRdFinal.write(listRdFinal)
jsonFileListRdFinal.close()

print("Total de leads duplicados RD: ", totalLeadsExctDupl)
print("Total de leads excluídos RD: ", totalDeletedLeads)
print("Finalizado")


## De/Para das Bases

> Aqui vamos fazer o de/para da base do RD Station para a base do Exact Sales e vice-versa

> **Total Leads Exact**: 27152

> **Total Leads RD**: 22822 (com pendência sobre o número total de leads via requisição)

### As bases

- Temos as duas bases de dados prontas, que foram extraídas via requisição na API de cada base
  - A base do RD Station foi tratada para ter no campo _custom_field_ ligado ao _CNPJ_ somente números, já que na extração original os dados vinham com caracteres especiais (,./-)
- Ambas as bases foram tratadas para que tenham sempre os 14 dígitos do CNPJ, tornando possível o abate das informações corretamente

### Procedimento

- Agora, vamos pesquisar para cada lead da base RD se ela existe na base Exact utilizando o método _put_ na requisição
  - **Se existe**: atualizar os dados que não estão preenchidos
  - **Se não existe**: criar o lead


In [ ]:
# Fazendo de/para da base exact/rd

# Lendo a base Exact
listExctOpen = open("exactListFinal.json")
listExct = json.load(listExctOpen)

# Lendo a base Rd
listRdOpen = open("completeDatabase_RD2.json")
listRd = json.load(listRdOpen)

totalLeadsExct = 27155
totalLeadsDePara = 0
totalLeadsNotDePara = 0

resultDePara = []
resultNotDePara = []

# Loop correndo para cada CNPJ da base exact e conferindo se existe na base rd
for s in range(0, len(listExct) - 1):
    print(s)  # 55
    for t in range(0, len(listExct[s])):
        CNPJExct = listExct[s][t]["cnpj"]
        for w in range(0, len(listRd)):
            for i in range(0, len(listRd[w])):
                for j in range(0, len(listRd[w][i]["custom_fields"]) - 1):
                    if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "CNPJ"):
                        res = {
                            k: v for k, v in listRd[w][i]["custom_fields"][j].items() if v == CNPJExct}

                        if (res):
                            resultDeParaObj = {
                                'CNPJ': CNPJExct,
                                'Nome Exact': listExct[s][t]["lead"],
                                'Nome RD': listRd[w][i]["name"],
                                'ID Exact': listExct[s][t]["id"],
                                'ID RD': listRd[w][i]["id"]
                            }
                            resultDePara.append(resultDeParaObj)

                            totalLeadsDePara = totalLeadsDePara + 1

resultDeParaStr = json.dumps(resultDePara)
jsonFileDePara = open("DePara.json", "w")
jsonFileDePara.write(resultDeParaStr)
jsonFileDePara.close()

print("Total de leads De/Para: ", totalLeadsDePara)
print("Finalizado")


# Atualizando Leads

> Atualizando leads no exact que já tem no RD

In [1]:
# Atualizando leads no exact que já tem no RD

import requests
import json

# Lendo arquivos
listOpen = open("DePara.json")
dePara = json.load(listOpen)

listOpen = open("databases/completeDatabase_RD2.json")
listRd = json.load(listOpen)

listOpen = open("databases/exactListFinal.json")
listExct = json.load(listOpen)

# Funções
# Função para a requisição da API


def request_API(values, id):
    url = 'https://api.exactspotter.com/v3/LeadsUpdate/' + str(id)
    headers = {
        'Content-Type': 'application/json',
        'token_exact': '35fc1bd4-7d6e-4f40-bffd-123d2f46054a'
    }

    result = requests.put(url, data=str(values), headers=headers)

    global status
    status = result.status_code

    print(result.status_code)
    # print(result.headers)
    # print(result.json())

    # return status


def replace_unicode(word):
    if (word == "None"):
        return ""
    else:
        word = word.replace("À", "\xc3\x80")
        word = word.replace("Á", "\xc3\x81")
        word = word.replace("Â", "\xc3\x82")
        word = word.replace("Ã", "\xc3\x83")
        word = word.replace("Ä", "\xc3\x84")
        word = word.replace("Å", "\xc3\x85")
        word = word.replace("Æ", "\xc3\x86")
        word = word.replace("Ç", "\xc3\x87")
        word = word.replace("È", "\xc3\x88")
        word = word.replace("É", "\xc3\x89")
        word = word.replace("Ê", "\xc3\x8a")
        word = word.replace("Ë", "\xc3\x8b")
        word = word.replace("Ì", "\xc3\x8c")
        word = word.replace("Í", "\xc3\x8d")
        word = word.replace("Î", "\xc3\x8e")
        word = word.replace("Ï", "\xc3\x8f")
        word = word.replace("Ð", "\xc3\x90")
        word = word.replace("Ñ", "\xc3\x91")
        word = word.replace("Ò", "\xc3\x92")
        word = word.replace("Ó", "\xc3\x93")
        word = word.replace("Ô", "\xc3\x94")
        word = word.replace("Õ", "\xc3\x95")
        word = word.replace("Ö", "\xc3\x96")
        word = word.replace("×", "\xc3\x97")
        word = word.replace("Ø", "\xc3\x98")
        word = word.replace("Ù", "\xc3\x99")
        word = word.replace("Ú", "\xc3\x9a")
        word = word.replace("Û", "\xc3\x9b")
        word = word.replace("Ü", "\xc3\x9c")
        word = word.replace("Ý", "\xc3\x9d")
        word = word.replace("Þ", "\xc3\x9e")
        word = word.replace("ß", "\xc3\x9f")
        word = word.replace("à", "\xc3\xa0")
        word = word.replace("á", "\xc3\xa1")
        word = word.replace("â", "\xc3\xa2")
        word = word.replace("ã", "\xc3\xa3")
        word = word.replace("ä", "\xc3\xa4")
        word = word.replace("å", "\xc3\xa5")
        word = word.replace("æ", "\xc3\xa6")
        word = word.replace("ç", "\xc3\xa7")
        word = word.replace("è", "\xc3\xa8")
        word = word.replace("é", "\xc3\xa9")
        word = word.replace("ê", "\xc3\xaa")
        word = word.replace("ë", "\xc3\xab")
        word = word.replace("ì", "\xc3\xac")
        word = word.replace("í", "\xc3\xad")
        word = word.replace("î", "\xc3\xae")
        word = word.replace("ï", "\xc3\xaf")
        word = word.replace("ð", "\xc3\xb0")
        word = word.replace("ñ", "\xc3\xb1")
        word = word.replace("ò", "\xc3\xb2")
        word = word.replace("ó", "\xc3\xb3")
        word = word.replace("ô", "\xc3\xb4")
        word = word.replace("õ", "\xc3\xb5")
        word = word.replace("ö", "\xc3\xb6")
        word = word.replace("÷", "\xc3\xb7")
        word = word.replace("ø", "\xc3\xb8")
        word = word.replace("ù", "\xc3\xb9")
        word = word.replace("ú", "\xc3\xba")
        word = word.replace("û", "\xc3\xbb")
        word = word.replace("ü", "\xc3\xbc")
        word = word.replace("ý", "\xc3\xbd")
        word = word.replace("þ", "\xc3\xbe")
        word = word.replace("ÿ", "\xc3\xbf")

        return word


# Loop correndo todos os leads do de/para buscando informação na base do rd e fazendo a requisição para inserir no exact
totalSuccess = 0
totalFailure = 0
finalSuccess = []
finalFailure = []
status = 0

for i in range(0, len(dePara)):
    print(i)  # 9464
    CNPJDePara = dePara[i]
    for s in range(0, len(listExct) - 1):
        for t in range(0, len(listExct[s])):
            # Pegando valores do exact
            CNPJExct = listExct[s][t]["cnpj"]
            IDExct = listExct[s][t]["id"]
            cel1Exact = listExct[s][t]["phone1"]
            addressExact = listExct[s][t]["street"]
            cityExact = listExct[s][t]["city"]
            stateExact = listExct[s][t]["state"]
            # districtExact = listExct[s][t]["district"]

            for w in range(0, len(listRd)):
                for i in range(0, len(listRd[w])):
                    for j in range(0, len(listRd[w][i]["custom_fields"]) - 1):
                        if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "CNPJ"):
                            res = {
                                k: v for k, v in listRd[w][i]["custom_fields"][j].items() if v == CNPJExct}
                        if (res):
                            if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "CELULAR 1"):
                                resCel1 = listRd[w][i]["custom_fields"][j]["value"]
                            if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "Endereço Completo"):
                                resAddress = listRd[w][i]["custom_fields"][j]["value"]
                            if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "CIDADE"):
                                resCity = listRd[w][i]["custom_fields"][j]["value"]
                            if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "UF"):
                                resState = listRd[w][i]["custom_fields"][j]["value"]
                            # if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "BAIRRO"):
                                # resDistrict = listRd[w][i]["custom_fields"][j]["value"]

                        # Se acha um CNPJ no RD, procura os dados pro objeto de requisição
                        if (res):
                            # Construindo objeto e fazendo requisição
                            try:
                                if (res and resCel1):
                                    # name = (listRd[w][i]["name"])

                                    phone = (str(resCel1))
                                    if (phone == 'None' or cel1Exact != ''):
                                        phone = (str(cel1Exact))

                                    address = (str(resAddress))
                                    if (address == 'None'):
                                        address = (str(addressExact))

                                    city = (str(resCity))
                                    if (city == 'None'):
                                        city = (str(cityExact))

                                    state = (str(resState))
                                    if (state == 'None'):
                                        state = (str(stateExact))

                                    # email = (str(listRd[w][i]["user"]["email"]))
                                    # website = (str(listRd[w][i]["url"]))
                                    cpfcnpj = (str(res["value"]))

                                    if (phone != 'None' or cel1Exact != ''):
                                        result = {
                                            "duplicityValidation": "true",
                                            "lead":
                                            {
                                                # "name": replace_unicode(name),
                                                # "sdrEmail": replace_unicode(email),
                                                "address": replace_unicode(address),
                                                "city": replace_unicode(city),
                                                "cpfcnpj": replace_unicode(cpfcnpj)
                                            }
                                        }

                                    else:
                                        result = {
                                            "duplicityValidation": "true",
                                            "lead":
                                            {
                                                # "name": replace_unicode(name),
                                                # "sdrEmail": replace_unicode(email),
                                                "phone": replace_unicode(phone),
                                                "address": replace_unicode(address),
                                                "city": replace_unicode(city),
                                                "cpfcnpj": replace_unicode(cpfcnpj)
                                            }
                                        }

                                    print(IDExct, result)
                                    input("Press Enter to continue...")

                                    # Requisição de PUT
                                    request_API(result, IDExct)

                                    if (status == 201):
                                        totalSuccess += 1
                                        finalSuccess.append(IDExct)
                                        print("Sucesso: ", totalSuccess)
                                    else:
                                        totalFailure += 1
                                        finalFailure.append(IDExct)
                                        print("Falha: ", totalFailure)

                                    resCel1 = ''
                                    resAddress = ''
                                    resCity = ''
                                    resState = ''
                                    resDistrict = ''

                                    name = ''
                                    email = ''
                                    phone = ''
                                    website = ''
                                    address = ''
                                    city = ''
                                    state = ''
                                    cpfcnpj = ''

                                    break

                            except:
                                continue

print("Finalizado")


0
8572822 {'duplicityValidation': 'true', 'lead': {'address': 'R TEODORO SAMPAIO 399 CONJ 41', 'city': 'SAO PAULO', 'cpfcnpj': '05805760000190'}}
201
{'@odata.context': 'http://api-v3.k8s.exactspotter.com/api/v3/$metadata#Edm.Int32', 'value': 8572822}
Sucesso:  1
8554946 {'duplicityValidation': 'true', 'lead': {'address': 'R JOAO ANTONIO DE OLIVEIRA 1.052 E 1.054', 'city': 'SAO PAULO', 'cpfcnpj': '03481155000185'}}
201
{'@odata.context': 'http://api-v3.k8s.exactspotter.com/api/v3/$metadata#Edm.Int32', 'value': 8554946}
Sucesso:  2
6499593 {'duplicityValidation': 'true', 'lead': {'address': 'R FORTUNATO FERRAZ 896 ', 'city': 'SAO PAULO', 'cpfcnpj': '65789117000141'}}
201
{'@odata.context': 'http://api-v3.k8s.exactspotter.com/api/v3/$metadata#Edm.Int32', 'value': 6499593}
Sucesso:  3
8555952 {'duplicityValidation': 'true', 'lead': {'address': 'R MARIA APARECIDA CARDIA, 182 - 190 192 204 224 172', 'city': 'SAO PAULO', 'cpfcnpj': '48070825000109'}}
201
{'@odata.context': 'http://api-v3.k8s

# Criando Leads Exact

> Criando leads que existem no rd e não existem no exact

In [ ]:
# Criando leads na exact

import requests
import json

# Lendo arquivos
listOpen = open("databases/completeDatabase_RD2.json")
listRd = json.load(listOpen)

listOpen = open("databases/exactListFinal.json")
listExct = json.load(listOpen)

# Funções
# Função para a requisição da API


def request_API(values, id):
    url = 'https://api.exactspotter.com/v3/LeadsAdd'
    headers = {
        'Content-Type': 'application/json',
        'token_exact': '35fc1bd4-7d6e-4f40-bffd-123d2f46054a'
    }

    result = requests.put(url, data=str(values), headers=headers)

    global status
    status = result.status_code

    print(result.status_code)
    # print(result.headers)
    # print(result.json())

    # return status


def replace_unicode(word):
    if (word == "None"):
        return ""
    else:
        word = word.replace("À", "\xc3\x80")
        word = word.replace("Á", "\xc3\x81")
        word = word.replace("Â", "\xc3\x82")
        word = word.replace("Ã", "\xc3\x83")
        word = word.replace("Ä", "\xc3\x84")
        word = word.replace("Å", "\xc3\x85")
        word = word.replace("Æ", "\xc3\x86")
        word = word.replace("Ç", "\xc3\x87")
        word = word.replace("È", "\xc3\x88")
        word = word.replace("É", "\xc3\x89")
        word = word.replace("Ê", "\xc3\x8a")
        word = word.replace("Ë", "\xc3\x8b")
        word = word.replace("Ì", "\xc3\x8c")
        word = word.replace("Í", "\xc3\x8d")
        word = word.replace("Î", "\xc3\x8e")
        word = word.replace("Ï", "\xc3\x8f")
        word = word.replace("Ð", "\xc3\x90")
        word = word.replace("Ñ", "\xc3\x91")
        word = word.replace("Ò", "\xc3\x92")
        word = word.replace("Ó", "\xc3\x93")
        word = word.replace("Ô", "\xc3\x94")
        word = word.replace("Õ", "\xc3\x95")
        word = word.replace("Ö", "\xc3\x96")
        word = word.replace("×", "\xc3\x97")
        word = word.replace("Ø", "\xc3\x98")
        word = word.replace("Ù", "\xc3\x99")
        word = word.replace("Ú", "\xc3\x9a")
        word = word.replace("Û", "\xc3\x9b")
        word = word.replace("Ü", "\xc3\x9c")
        word = word.replace("Ý", "\xc3\x9d")
        word = word.replace("Þ", "\xc3\x9e")
        word = word.replace("ß", "\xc3\x9f")
        word = word.replace("à", "\xc3\xa0")
        word = word.replace("á", "\xc3\xa1")
        word = word.replace("â", "\xc3\xa2")
        word = word.replace("ã", "\xc3\xa3")
        word = word.replace("ä", "\xc3\xa4")
        word = word.replace("å", "\xc3\xa5")
        word = word.replace("æ", "\xc3\xa6")
        word = word.replace("ç", "\xc3\xa7")
        word = word.replace("è", "\xc3\xa8")
        word = word.replace("é", "\xc3\xa9")
        word = word.replace("ê", "\xc3\xaa")
        word = word.replace("ë", "\xc3\xab")
        word = word.replace("ì", "\xc3\xac")
        word = word.replace("í", "\xc3\xad")
        word = word.replace("î", "\xc3\xae")
        word = word.replace("ï", "\xc3\xaf")
        word = word.replace("ð", "\xc3\xb0")
        word = word.replace("ñ", "\xc3\xb1")
        word = word.replace("ò", "\xc3\xb2")
        word = word.replace("ó", "\xc3\xb3")
        word = word.replace("ô", "\xc3\xb4")
        word = word.replace("õ", "\xc3\xb5")
        word = word.replace("ö", "\xc3\xb6")
        word = word.replace("÷", "\xc3\xb7")
        word = word.replace("ø", "\xc3\xb8")
        word = word.replace("ù", "\xc3\xb9")
        word = word.replace("ú", "\xc3\xba")
        word = word.replace("û", "\xc3\xbb")
        word = word.replace("ü", "\xc3\xbc")
        word = word.replace("ý", "\xc3\xbd")
        word = word.replace("þ", "\xc3\xbe")
        word = word.replace("ÿ", "\xc3\xbf")

        return word


# Loop correndo todos os leads do de/para buscando informação na base do rd e fazendo a requisição para inserir no exact
totalSuccess = 0
totalFailure = 0
finalSuccess = []
finalFailure = []
status = 0

# Loop Rd
for w in range(0, len(listRd)):
    for i in range(0, len(listRd[w])):
        for j in range(0, len(listRd[w][i]["custom_fields"]) - 1):
            # Loop Exact
            for s in range(0, len(listExct) - 1):
                for t in range(0, len(listExct[s])):
                    if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "CNPJ"):
                        res = {
                            k: v for k, v in listRd[w][i]["custom_fields"][j].items() if v == CNPJExct}
                    if (res):
                        if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "CELULAR 1"):
                            resCel1 = listRd[w][i]["custom_fields"][j]["value"]
                        if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "Endereço Completo"):
                            resAddress = listRd[w][i]["custom_fields"][j]["value"]
                        if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "CIDADE"):
                            resCity = listRd[w][i]["custom_fields"][j]["value"]
                        if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "UF"):
                            resState = listRd[w][i]["custom_fields"][j]["value"]
                        if (listRd[w][i]["custom_fields"][j]["custom_field"]["label"] == "BAIRRO"):
                            resDistrict = listRd[w][i]["custom_fields"][j]["value"]

                    # Se acha um CNPJ no RD, procura os dados pro objeto de requisição
                        if (res):
                            # Construindo objeto e fazendo requisição
                            try:
                                if (res and resCel1):
                                    # name = (listRd[w][i]["name"])

                                    phone = (str(resCel1))
                                    if (phone == 'None' or cel1Exact != ''):
                                        phone = (str(cel1Exact))

                                    address = (str(resAddress))
                                    if (address == 'None'):
                                        address = (str(addressExact))

                                    city = (str(resCity))
                                    if (city == 'None'):
                                        city = (str(cityExact))

                                    state = (str(resState))
                                    if (state == 'None'):
                                        state = (str(stateExact))

                                    # email = (str(listRd[w][i]["user"]["email"]))
                                    # website = (str(listRd[w][i]["url"]))
                                    cpfcnpj = (str(res["value"]))

                                    if (phone != 'None' or cel1Exact != ''):
                                        result = {
                                            "duplicityValidation": "true",
                                            "lead":
                                            {
                                                # "name": replace_unicode(name),
                                                # "sdrEmail": replace_unicode(email),
                                                "address": replace_unicode(address),
                                                "city": replace_unicode(city),
                                                "cpfcnpj": replace_unicode(cpfcnpj)
                                            }
                                        }

                                    else:
                                        result = {
                                            "duplicityValidation": "true",
                                            "lead":
                                            {
                                                # "name": replace_unicode(name),
                                                # "sdrEmail": replace_unicode(email),
                                                "phone": replace_unicode(phone),
                                                "address": replace_unicode(address),
                                                "city": replace_unicode(city),
                                                "cpfcnpj": replace_unicode(cpfcnpj)
                                            }
                                        }

                                    print(IDExct, result)
                                    input("Press Enter to continue...")

                                    # Requisição de PUT
                                    request_API(result, IDExct)

                                    if (status == 201):
                                        totalSuccess += 1
                                        finalSuccess.append(IDExct)
                                        print("Sucesso: ", totalSuccess)
                                    else:
                                        totalFailure += 1
                                        finalFailure.append(IDExct)
                                        print("Falha: ", totalFailure)

                                    resCel1 = ''
                                    resAddress = ''
                                    resCity = ''
                                    resState = ''
                                    resDistrict = ''

                                    name = ''
                                    email = ''
                                    phone = ''
                                    website = ''
                                    address = ''
                                    city = ''
                                    state = ''
                                    cpfcnpj = ''

                                    break

                            except:
                                continue

print('Finalizado')